In [ ]:
# ============================================================
# DYNAMIC VRPP — 30-DAY SIMULATION WITH TRACEABILITY
# CORRECTED VERSION: Overflow is accounted at the START of each day
# ============================================================

import time

import gurobipy as gp
import pandas as pd
from gurobipy import GRB

# --------------------------
# User files
file_parameters = r"../data/wsr_simulator/daily_waste/parameters.xlsx"
file_distances = r"../data/wsr_simulator/distance_matrix/distance_104bins.xlsx"

output_excel_kpi = r"../assets/output/31_days/riomaior_104/TestVRPP-104bins_KPI_DYNAMIC_final_ajustado.xlsx"
output_excel_flows = r"../assets/output/31_days/riomaior_104/TestVRPP-104bins_Flows_DYNAMIC_final_ajustado.xlsx"
output_excel_dynamic = output_excel_kpi.replace("_KPI_", "_DYNAMIC_")
output_excel_risk = output_excel_kpi.replace("_KPI_", "_OVERFLOW_RISK_TRACE_")

# --------------------------
# Model parameters
C = 1
R = 0.5837
Omega = 0.1
Q = 3500
delta = 0
T_days = 30  # set to 1 for quick test; change to 30 for full sim
T = list(range(1, T_days + 1))
B = 19
vol = 2.5

params = {"OutputFlag": 0}
env = gp.Env(params=params)

# --------------------------
# Read parameters
df_params = pd.read_excel(file_parameters, sheet_name="Si_Ei", engine="openpyxl")
IDs_model = df_params["ID"].astype(int).tolist()
S_base = {int(r["ID"]): float(r["Si"]) for _, r in df_params.iterrows()}
E_dict = {int(r["ID"]): float(r["Ei"]) for _, r in df_params.iterrows()}
avg_ai = {int(r["ID"]): float(r["Mean"]) for _, r in df_params.iterrows()}

# Read daily accumulation percentages
df_ait = pd.read_excel(file_parameters, sheet_name="%ai_t", engine="openpyxl")
S_init = {}
ait_kg_dict = {}
for _, row in df_ait.iterrows():
    i = int(row["ID"])
    percent_S0 = float(row["Day 0"]) if "Day 0" in df_ait.columns and not pd.isna(row["Day 0"]) else 0.0
    S_init[i] = (percent_S0 / 100.0) * B * vol
    for t in T:
        col = f"Day {t}"
        ait_kg_dict[(i, t)] = (
            (float(row[col]) / 100.0 * B * vol) if col in df_ait.columns and not pd.isna(row[col]) else 0.0
        )

# --------------------------
# Distance matrix
df_dist = pd.read_excel(file_distances, engine="openpyxl", index_col=0)
df_dist.index = df_dist.index.astype(int)
df_dist.columns = df_dist.columns.astype(int)

nodes_real = IDs_model
n_bins = len(nodes_real)
fictitious_node = n_bins + 1
nodes = [0] + nodes_real + [fictitious_node]

distances = {}
for i in nodes:
    for j in nodes:
        ii = 0 if i == fictitious_node else i
        jj = 0 if j == fictitious_node else j
        distances[(i, j)] = float(df_dist.loc[ii, jj])

# --------------------------
# Initialize dynamic storage
S_current = {i: S_base[i] for i in nodes_real}

# --------------------------
# Tables for export
dynamic_rows = []
kpi_rows = []
flow_rows = []
overflow_risk_rows = []

# ==========================
# DAILY SIMULATION LOOP
# ==========================
for t in T:
    print(f"\n=== Day {t} ===")

    # --------------------------
    # STEP 1: Calculate total waste attempted to be deposited
    Wit_before_dict = {i: S_current[i] + ait_kg_dict.get((i, t), 0.0) for i in nodes_real}

    # --------------------------
    # STEP 2: OVERFLOW OCCURS IMMEDIATELY (physical reality)
    # Collectable = what fits in the container (limited by Ei)
    # Overflow = what spills over (lost immediately, regardless of collection)
    Wit_collectable_dict = {i: min(Wit_before_dict[i], E_dict[i]) for i in nodes_real}
    Wit_overflow_initial_dict = {i: max(0.0, Wit_before_dict[i] - E_dict[i]) for i in nodes_real}

    # Count total overflow at the start of the day (DYNAMIC ACCOUNTING)
    total_overflow_initial_day = sum(Wit_overflow_initial_dict.values())

    print(f"Total overflow at start of day (before collection decision): {total_overflow_initial_day:.2f} kg")

    # --------------------------
    # STEP 3: Determine planned collection (overflow risk) based on S_current + 2*avg_ai >= Ei
    collected_planned_dict = {i: 1 if S_current[i] + 2 * avg_ai[i] >= E_dict[i] else 0 for i in nodes_real}
    planned_bins = [i for i, v in collected_planned_dict.items() if v == 1]
    print("S_current:", S_current)
    print("avg_ai:", avg_ai)
    print("E_dict:", E_dict)
    print(f"Planned bins to collect (based on S_current + 2*avg_ai >= Ei): {planned_bins}")

    # --------------------------
    # Trace overflow risk table (based on S_current + 2*avg_ai)
    for i in nodes_real:
        S_now = S_current[i]
        avg_term_dupl = 2 * avg_ai[i]
        risk_value = S_now + avg_term_dupl
        overflow_risk_rows.append(
            {
                "Day": t,
                "ID": i,
                "S_current": S_now,
                "2x_avg_ai": avg_term_dupl,
                "S_current+2avg": risk_value,
                "Ei": E_dict[i],
                "Margin_to_Ei": E_dict[i] - risk_value,
                "Wit_before": Wit_before_dict[i],
                "Overflow_initial": Wit_overflow_initial_dict[i],
                "RiskStatus": "RISK" if risk_value >= E_dict[i] else "SAFE",
            }
        )

    print(f"Planned bins to collect (based on S_current + 2*avg_ai >= Ei): {planned_bins}")

    # --------------------------
    # Skip optimization if no overflow risk
    if len(planned_bins) == 0:
        print("No overflow risk, skipping optimization.")
        for i in nodes_real:
            S_before = S_current[i]
            a_it = ait_kg_dict.get((i, t), 0.0)
            Wit_before = Wit_before_dict[i]
            Wit_collectable = Wit_collectable_dict[i]
            overflow_initial = Wit_overflow_initial_dict[i]

            collected_planned = 0
            collected_actual = 0

            # CORRECCIÓN CRÍTICA: Sin recolección, el contenedor queda con lo que cabe (Ei max)
            # El overflow ya ocurrió al inicio del día
            S_after = Wit_collectable  # min(Wit_before, Ei)
            kg_collected_actual = 0.0
            kg_overflow_actual = overflow_initial  # Ya se perdió al inicio

            profit_before = 0.0
            profit_after = 0.0

            dynamic_rows.append(
                {
                    "Day": t,
                    "ID": i,
                    "S_before": S_before,
                    "a_it": a_it,
                    "Wit_before": Wit_before,
                    "Wit_collectable": Wit_collectable,
                    "Overflow_initial": overflow_initial,
                    "Collected_planned": collected_planned,
                    "Collected_actual": collected_actual,
                    "Kg_collected_actual": kg_collected_actual,
                    "Kg_overflow_actual": kg_overflow_actual,
                    "Wit_after": S_after,
                    "Profit_before": profit_before,
                    "Profit_after": profit_after,
                    "RiskStatus": "SAFE",
                }
            )
            S_current[i] = S_after

            print(
                f"Bin {i} | S_before={S_before:.2f} | a_it={a_it:.2f} | Wit_before={Wit_before:.2f} | "
                f"Collectable={Wit_collectable:.2f} | Overflow_initial={overflow_initial:.2f} | "
                f"Collected_actual={collected_actual} | Wit_after={S_after:.2f}"
            )

        # KPI for day with no collection
        kpi_rows.append(
            {
                "Day": t,
                "KgCollected": 0.0,
                "KgOverflow": total_overflow_initial_day,
                "NumBinsOverflow_All": sum(1 for v in Wit_overflow_initial_dict.values() if v > 0),
                "IDsBinsOverflow_All": [i for i in nodes_real if Wit_overflow_initial_dict[i] > 0],
                "Distance": 0.0,
                "Profit": 0.0,
                "VehicleCost": 0.0,
                "FO": 0.0,
                "NumVehicles": 0,
                "BinsVisited": 0,
                "IDsBinsVisited": [],
                "BinsNotVisited": len(nodes_real),
                "IDsBinsNotVisited": nodes_real,
                "Routes": [],
                "GapPercent": None,
                "RatioKgPerKm": None,
                "Observation": "No overflow risk - skipped optimization",
                "SolveTime_s": 0.0,
            }
        )
        continue

    # --------------------------
    # Build optimization model
    model = gp.Model(f"VRP_Day_{t}", env=env)
    model.Params.LogToConsole = 0
    x = model.addVars(nodes, nodes, vtype=GRB.BINARY, name="x")
    g = model.addVars(nodes_real, vtype=GRB.BINARY, name="g")
    f = model.addVars(nodes, nodes, vtype=GRB.CONTINUOUS, name="f")
    model.Params.TimeLimit = 28800
    model.Params.MIPGap = 0.000

    # --------------------------
    # Objective: based on COLLECTABLE weight (limited by Ei)
    model.setObjective(
        R * gp.quicksum(Wit_collectable_dict[i] * g[i] for i in nodes_real)
        - C * gp.quicksum(x[i, j] * distances[(i, j)] for i in nodes for j in nodes if i != j)
        - Omega * gp.quicksum(x[0, j] for j in nodes_real),
        GRB.MAXIMIZE,
    )

    # --------------------------
    # Constraints
    model.addConstrs(
        (g[i] == 0 for i in nodes_real if Wit_collectable_dict[i] == 0),
        name="no_collect_empty",
    )

    model.addConstrs(
        (gp.quicksum(x[i, j] for i in nodes if i != fictitious_node and i != j) == g[j] for j in nodes_real),
        name="visit_if_collect",
    )

    model.addConstrs((g[i] <= 1 for i in nodes_real), name="collect_at_most_once")

    model.addConstr(
        gp.quicksum(g[i] for i in planned_bins) >= len(planned_bins) - n_bins * delta,
        name="min_planned_collection",
    )

    model.addConstr(
        gp.quicksum(x[0, j] for j in nodes if j != 0) - gp.quicksum(x[i, 0] for i in nodes if i != fictitious_node)
        == 0,
        name="depot_balance",
    )

    model.addConstrs(
        (
            gp.quicksum(x[i, h] for i in nodes if i != fictitious_node) - gp.quicksum(x[h, j] for j in nodes if j != 0)
            == 0
            for h in nodes_real
        ),
        name="flow_conservation",
    )

    # Flow constraints based on COLLECTABLE weight (limited by Ei)
    model.addConstrs(
        (f[i, j] <= Q * x[i, j] for i in nodes for j in nodes if i != j),
        name="capacity_link",
    )

    model.addConstrs(
        (
            gp.quicksum(f[i, j] for j in nodes if j != 0) - gp.quicksum(f[j, i] for j in nodes if j != fictitious_node)
            == Wit_collectable_dict[i] * g[i]
            for i in nodes_real
        ),
        name="flow_balance",
    )

    # --------------------------
    # Solve
    start = time.time()
    model.optimize()
    solve_time = time.time() - start

    # --------------------------
    # Update dynamic S and record daily table
    print("\nDay summary (per bin after optimization):")
    total_profit_day = 0.0
    total_kg_collected_day = 0.0
    total_kg_overflow_day = 0.0

    for i in nodes_real:
        S_before = S_current[i]
        a_it = ait_kg_dict.get((i, t), 0.0)
        Wit_before = Wit_before_dict[i]
        Wit_collectable = Wit_collectable_dict[i]
        overflow_initial = Wit_overflow_initial_dict[i]

        collected_planned = collected_planned_dict[i]
        collected_actual = 1 if g[i].X > 0.5 else 0

        # CORRECCIÓN CRÍTICA: El overflow ya ocurrió al inicio, independiente de la recolección
        kg_overflow_actual = overflow_initial  # Siempre se pierde al inicio del día

        if collected_actual == 1:
            # Se recolecta lo que cabe en el contenedor
            kg_collected_actual = Wit_collectable
            S_after = 0.0  # Contenedor vacío
        else:
            # NO se recolecta: el contenedor queda con lo que cabe (el overflow ya se perdió)
            kg_collected_actual = 0.0
            S_after = Wit_collectable  # min(Wit_before, Ei)

        # Profits
        profit_before = R * Wit_collectable if collected_planned == 1 else 0.0
        profit_after = R * kg_collected_actual

        risk_status = "RISK" if S_before + 2 * avg_ai[i] >= E_dict[i] else "SAFE"

        dynamic_rows.append(
            {
                "Day": t,
                "ID": i,
                "S_before": S_before,
                "a_it": a_it,
                "Wit_before": Wit_before,
                "Wit_collectable": Wit_collectable,
                "Overflow_initial": overflow_initial,
                "Collected_planned": collected_planned,
                "Collected_actual": collected_actual,
                "Kg_collected_actual": kg_collected_actual,
                "Kg_overflow_actual": kg_overflow_actual,
                "Wit_after": S_after,
                "Profit_before": profit_before,
                "Profit_after": profit_after,
                "RiskStatus": risk_status,
            }
        )

        S_current[i] = S_after
        total_profit_day += profit_after
        total_kg_collected_day += kg_collected_actual
        total_kg_overflow_day += kg_overflow_actual

        print(
            f"Bin {i} | S_before={S_before:.2f} | a_it={a_it:.2f} | Wit_before={Wit_before:.2f} | "
            f"Collectable={Wit_collectable:.2f} | Overflow={overflow_initial:.2f} | "
            f"Collected={collected_actual} | Kg_collected={kg_collected_actual:.2f} | "
            f"Wit_after={S_after:.2f} | Profit={profit_after:.2f} | Risk={risk_status}"
        )

    # --------------------------
    # Export flows (x, g)
    for i in nodes:
        for j in nodes:
            if i != j and hasattr(x[i, j], "X") and x[i, j].X > 0.5:
                flow_rows.append({"Day": t, "Var": "x", "i": i, "j": j, "Value": x[i, j].X})
    for i in nodes_real:
        if hasattr(g[i], "X") and g[i].X > 0.5:
            flow_rows.append({"Day": t, "Var": "g", "i": i, "j": "-", "Value": g[i].X})

    # --------------------------
    # KPI and route calculation
    active_arcs = [(i, j) for i in nodes for j in nodes if i != j and hasattr(x[i, j], "X") and x[i, j].X > 0.5]
    starts = [j for (i, j) in active_arcs if i == 0 and j != 0]

    vehicles_routes = []
    used_edges = set()
    for s in starts:
        route = [0, s]
        used_edges.add((0, s))
        cur = s
        loop_guard = 0
        while True:
            loop_guard += 1
            if loop_guard > 2000:
                break
            next_candidates = [b for (a, b) in active_arcs if a == cur and (a, b) not in used_edges]
            if not next_candidates:
                next_candidates = [b for (a, b) in active_arcs if a == cur]
            if not next_candidates:
                break
            nxt = next_candidates[0]
            if nxt == 0:
                break
            route.append(nxt)
            used_edges.add((cur, nxt))
            cur = nxt
            if cur == fictitious_node:
                break
        if route[-1] != fictitious_node and (route[-1], fictitious_node) in [(a, b) for (a, b) in active_arcs]:
            route.append(fictitious_node)
        if len(route) >= 2 and route[0] == 0:
            vehicles_routes.append(route)

    bins_visited = [i for i in nodes_real if hasattr(g[i], "X") and g[i].X > 0.5]
    bins_not_visited = [i for i in nodes_real if i not in bins_visited]
    print("Vehicle routes:", vehicles_routes)

    # Bins in overflow at start of day
    bins_overflow_all = [i for i in nodes_real if Wit_overflow_initial_dict[i] > 0]

    total_dist = sum(
        distances.get((i, j), 0.0) * (x[i, j].X if hasattr(x[i, j], "X") else 0.0)
        for i in nodes
        for j in nodes
        if i != j
    )

    kg_collected = total_kg_collected_day
    kg_overflow = total_kg_overflow_day  # Total overflow from start of day

    profit_total = R * kg_collected
    vehicle_cost = Omega * len(vehicles_routes)
    FO = profit_total - C * total_dist - vehicle_cost

    model_gap = model.MIPGap if hasattr(model, "MIPGap") else None
    gap_percent = (model_gap * 100.0) if model_gap is not None else None
    ratio_kg_per_km = (kg_collected / total_dist) if total_dist > 0 else None

    obs = []
    bad_routes = [r for r in vehicles_routes if r[0] != 0 or r[-1] != fictitious_node]
    if bad_routes:
        obs.append(f"Routes do not correctly end at fictitious_node: {len(bad_routes)}")

    kpi_rows.append(
        {
            "Day": t,
            "KgCollected": kg_collected,
            "KgOverflow": kg_overflow,
            "NumBinsOverflow_All": len(bins_overflow_all),
            "IDsBinsOverflow_All": bins_overflow_all,
            "Distance": total_dist,
            "Profit": profit_total,
            "VehicleCost": vehicle_cost,
            "FO": FO,
            "NumVehicles": len(vehicles_routes),
            "BinsVisited": len(bins_visited),
            "IDsBinsVisited": bins_visited,
            "BinsNotVisited": len(bins_not_visited),
            "IDsBinsNotVisited": bins_not_visited,
            "Routes": vehicles_routes,
            "GapPercent": gap_percent,
            "RatioKgPerKm": ratio_kg_per_km,
            "Observation": "; ".join(obs),
            "SolveTime_s": solve_time,
        }
    )

# --------------------------
# EXPORT TABLES
pd.DataFrame(dynamic_rows).to_excel(output_excel_dynamic, index=False)
pd.DataFrame(kpi_rows).to_excel(output_excel_kpi, index=False)
pd.DataFrame(flow_rows).to_excel(output_excel_flows, index=False)
pd.DataFrame(overflow_risk_rows).to_excel(output_excel_risk, index=False)

print("\n" + "=" * 60)
print("✅ Dynamic simulation completed successfully")
print(f"✅ Daily bin summary exported -> {output_excel_dynamic}")
print(f"✅ KPI exported -> {output_excel_kpi}")
print(f"✅ Flows exported -> {output_excel_flows}")
print(f"✅ Overflow risk trace exported -> {output_excel_risk}")
print("=" * 60)


=== Day 1 ===
Total overflow at start of day (before collection decision): 0.00 kg
S_current: {192: 17.575, 197: 37.525000000000006, 629: 0.0, 754: 20.425, 787: 29.45, 892: 38.949999999999996, 904: 13.3, 991: 17.575, 995: 3.325, 1171: 21.375, 1172: 32.300000000000004, 1173: 38.949999999999996, 1298: 25.175, 1410: 38.0, 1459: 38.949999999999996, 1606: 38.949999999999996, 1610: 4.275, 1621: 38.949999999999996, 1655: 38.949999999999996, 3224: 38.949999999999996, 3315: 30.4, 3317: 13.774999999999999, 5673: 0.0, 5730: 13.774999999999999, 5752: 38.949999999999996, 5815: 1.9, 5931: 24.224999999999998, 5933: 30.4, 5938: 13.774999999999999, 5977: 16.625, 6046: 18.05, 6092: 38.949999999999996, 6225: 38.949999999999996, 6228: 14.25, 6237: 0.0, 6240: 30.875, 6904: 35.15, 6911: 38.949999999999996, 7245: 24.224999999999998, 7445: 38.949999999999996, 7702: 0.0, 7733: 38.949999999999996, 10799: 36.575, 10835: 17.575, 11213: 36.1, 11554: 18.05, 11555: 25.650000000000006, 11558: 12.825000000000003, 115

OSError: Cannot save file into a non-existent directory: '..assets/output/31_days/riomaior_104'